In [ ]:
from pyairtable import Api
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm

# Set up your Personal Access Token, Base ID, and table name
personal_access_token = 'patkM1lr1Dce72438.59d244e3b0ec30a3e2a7db799ea86830fc86a133fc6d484063ea4291cd758500' # readonly_python_token
# base_id = 'appN1iJxfFFRYTbYX'  #CCEMP Base
base_id = 'appJcfRoSFT5Daikk' # Events Base
table_name = 'Stormwater Seminar'  # Table Name
# Initialize the API with the token
api = Api(personal_access_token)
table = api.table(base_id, table_name)
records = table.all()

# Print the FirstName, LastName, and Company fields from each record
for record in records:
    first_name = record['fields'].get('FirstName')
    last_name = record['fields'].get('LastName')
    company = record['fields'].get('Company')
    print(f"First Name: {first_name}, Last Name: {last_name}, Company: {company}")


In [ ]:

# Convert the records to a list of dictionaries
data = []
for record in records:
    fields = record['fields']
    data.append({
        'First Name': fields.get('FirstName', ''),
        'Last Name': fields.get('LastName', ''),
        'Company/Organization': fields.get('Company', ''),
        'createdTime': record.get('createdTime', '')
    })

# Replace None with empty strings and sort the data by 'Last Name'
data = [{k: (v if v is not None else '') for k, v in d.items()} for d in data]
data.sort(key=lambda x: x['createdTime'])

# Create a PDF canvas
output_pdf = 'name_tags_{}.pdf'.format(table_name )
c = canvas.Canvas(output_pdf, pagesize=letter)
width, height = letter

# Define the size of each name tag (9 cm x 5 cm)
tag_width = 9 * cm
tag_height = 5 * cm

# Define margins and spacing
margin_x = 0.5 * cm
margin_y = 0.5 * cm
spacing_x = 0.5 * cm
spacing_y = 0.5 * cm

# Calculate the number of tags that fit per row and column (2x4 layout)
tags_per_row = 2
tags_per_col = 5

# Define the starting position
x_start = margin_x
y_start = height - margin_y - tag_height

# Calculate total tags including blank tags for the last page and two additional pages
total_tags = len(data) + (tags_per_row * tags_per_col - (len(data) % (tags_per_row * tags_per_col))) + (tags_per_row * tags_per_col * 2)

# Loop through each attendee and create a name tag
for index in range(total_tags):
    if index > 0 and index % (tags_per_row * tags_per_col) == 0:
        c.showPage()
        y_start = height - margin_y - tag_height

    x = x_start + (index % tags_per_row) * (tag_width + spacing_x)
    y = y_start - (index // tags_per_row % tags_per_col) * (tag_height + spacing_y)

    # Draw the border of the name tag
    c.rect(x, y, tag_width, tag_height)

    # Add the association name
    c.setFont("Helvetica-Bold", 10)
    c.drawString(x + 0.2 * cm, y + tag_height - 0.5 * cm, "CCEMP - Association of Chinese Canadian")
    c.drawString(x + 0.2 * cm, y + tag_height - 0.9 * cm, "Environmental & Municipal Professionals")

    if index < len(data):
        row = data[index]
        # Add the attendee's name
        c.setFont("Helvetica-Bold", 24)
        name_text = f"{row['First Name']} {row['Last Name']}"
        name_width = c.stringWidth(name_text, "Helvetica-Bold", 24)
        if name_width > tag_width - 0.6 * cm:
            # Reduce font size if name is too wide
            c.setFont("Helvetica-Bold", 20)
            name_text = f"{row['First Name']} {row['Last Name']}"
            name_width = c.stringWidth(name_text, "Helvetica-Bold", 20)
        c.drawString(x + 0.2 * cm, y + tag_height - 2.5 * cm, name_text)

        # Add the attendee's company/organization
        c.setFont("Helvetica", 16.5)
        org_text = row['Company/Organization']
        org_width = c.stringWidth(org_text, "Helvetica", 16.5)
        if org_width > tag_width - 0.6 * cm:
            # Reduce font size if organization is too wide
            c.setFont("Helvetica", 12)
            org_text = row['Company/Organization']
            org_width = c.stringWidth(org_text, "Helvetica", 12)
        c.drawString(x + 0.2 * cm, y + tag_height - 3.7 * cm, org_text)
    else:
        # Add placeholder for blank tags
        c.setFont("Helvetica-Bold", 24)
        c.drawString(x + 0.2 * cm, y + tag_height - 2.5 * cm, " ")

        # Add the attendee's company/organization
        c.setFont("Helvetica", 16.5)
        c.drawString(x + 0.2 * cm, y + tag_height - 3.7 * cm, " ")

# Save the PDF
c.save()


In [ ]:
from collections import defaultdict

# Sort the records by last name alphabetically
sorted_records = sorted(records, key=lambda r: r['fields'].get('LastName', '').upper())

# Calculate group size and initialize variables
total_groups = 8
people_per_group = max(1, len(sorted_records) // total_groups)
data = []
group_ranges = defaultdict(list)
group_index, start_index = 1, 0

# Assign sorted records to groups
for i, record in enumerate(sorted_records):
    fields = record['fields']
    last_name = fields.get('LastName', '')

    if last_name:
        # Collect initials and assign to group
        first_letter = last_name[0].upper()
        group_ranges[group_index].append(first_letter)
        data.append({
            'First Name': fields.get('FirstName', ''),
            'Last Name': last_name,
            'Company/Organization': fields.get('Company', ''),
            'createdTime': record.get('createdTime', ''),
            'Group': group_index
        })

    # Switch to next group when group is filled and last name initials change
    if (i - start_index + 1) >= people_per_group and group_index < total_groups:
        next_last_name = sorted_records[i + 1]['fields'].get('LastName', '') if i + 1 < len(sorted_records) else ''
        if not next_last_name or next_last_name[0].upper() != last_name[0].upper():
            start_index, group_index = i + 1, group_index + 1

# Print group summaries
for group, initials in sorted(group_ranges.items()):
    print(f"Group {group} ({', '.join(sorted(set(initials)))}): {len([d for d in data if d['Group'] == group])} people")


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from collections import defaultdict

def create_group_tags_pdf(sorted_records, pdf_filename="group_tags.pdf"):
    # Sort the records by last name alphabetically
    sorted_records = sorted(records, key=lambda r: r['fields'].get('LastName', '').upper())

    # Calculate group size and initialize variables
    total_groups = 8
    people_per_group = max(1, len(sorted_records) // total_groups)
    group_ranges = defaultdict(list)
    data = []
    group_index, start_index = 1, 0

    # Assign sorted records to groups
    for i, record in enumerate(sorted_records):
        fields = record['fields']
        last_name = fields.get('LastName', '')

        if last_name:
            first_letter = last_name[0].upper()
            group_ranges[group_index].append(first_letter)
            data.append({
                'First Name': fields.get('FirstName', ''),
                'Last Name': last_name,
                'Company/Organization': fields.get('Company', ''),
                'createdTime': record.get('createdTime', ''),
                'Group': group_index
            })

        # Switch to the next group when necessary
        if (i - start_index + 1) >= people_per_group and group_index < total_groups:
            next_last_name = sorted_records[i + 1]['fields'].get('LastName', '') if i + 1 < len(sorted_records) else ''
            if not next_last_name or next_last_name[0].upper() != last_name[0].upper():
                start_index, group_index = i + 1, group_index + 1

    # Create a PDF and set up a canvas
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter

    # Set up styles for the tags
    tag_width = width // 2 - 40  # Tag width to fit 2 tags per row
    tag_height = 120  # Fixed height for each tag
    margin = 20
    y_position = height - margin

    # Generate group tags
    for group, initials in sorted(group_ranges.items()):
        initials_str = ', '.join(sorted(set(initials)))
        group_size = len([d for d in data if d['Group'] == group])

        # Draw a rectangle for the tag
        c.setFillColor(colors.whitesmoke)
        c.setLineWidth(3) 
        c.rect(margin, y_position - tag_height, tag_width, tag_height, fill=True, stroke=True)

        # Add group info to the tag
        c.setFillColor(colors.black)
        c.setFont("Helvetica-Bold", 14)
        c.drawString(margin + 10, y_position - 40, f"Group {group}")
        c.setFont("Helvetica", 16)
        c.drawString(margin + 10, y_position - 60, f"Last Name: {initials_str}")
        c.setFont("Helvetica", 10)
        c.drawString(margin + 10, y_position - 80, f"People: {group_size}")

        # Move to the next tag position
        margin += tag_width + 40  # Spacing between tags
        if margin + tag_width > width:  # Move to next row
            margin = 20
            y_position -= tag_height + 20

        # Check if we need to add a new page
        if y_position < margin + tag_height:
            c.showPage()  # Create a new page
            y_position = height - margin

    # Save the PDF
    c.save()

# Sample usage with `sorted_records` passed in
create_group_tags_pdf(sorted_records)
